In [7]:
from langchain.retrievers import EnsembleRetriever, BM25Retriever
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_core.documents import Document
from dotenv import load_dotenv

load_dotenv()

True

In [15]:
%pip install rank_bm25


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [8]:
# ──────────────────────────────────────────────────────────────────
# SETUP: Create our sample company data
# ──────────────────────────────────────────────────────────────────

chunks = [
    "Microsoft acquired GitHub for 7.5 billion dollars in 2018.",
    "Tesla Cybertruck production ramp begins in 2024.",
    "Google is a large technology company with global operations.",
    "Tesla reported strong quarterly results. Tesla continues to lead in electric vehicles. Tesla announced new manufacturing facilities.",
    "SpaceX develops Starship rockets for Mars missions.",
    "The tech giant acquired the code repository platform for software development.",
    "NVIDIA designs Starship architecture for their new GPUs.",
    "Tesla Tesla Tesla financial quarterly results improved significantly.",
    "Cybertruck reservations exceeded company expectations.",
    "Microsoft is a large technology company with global operations."
]

In [9]:
# Convert to Document objects for LangChain
documents = [Document(page_content=chunk, metadata={"source": f"chunk_{i}"}) for i, chunk in enumerate(chunks)]

print("Sample Data:")
for i, chunk in enumerate(chunks, 1):
    print(f"{i}. {chunk}")

print("\n" + "="*80)

Sample Data:
1. Microsoft acquired GitHub for 7.5 billion dollars in 2018.
2. Tesla Cybertruck production ramp begins in 2024.
3. Google is a large technology company with global operations.
4. Tesla reported strong quarterly results. Tesla continues to lead in electric vehicles. Tesla announced new manufacturing facilities.
5. SpaceX develops Starship rockets for Mars missions.
6. The tech giant acquired the code repository platform for software development.
7. NVIDIA designs Starship architecture for their new GPUs.
8. Tesla Tesla Tesla financial quarterly results improved significantly.
9. Cybertruck reservations exceeded company expectations.
10. Microsoft is a large technology company with global operations.



 
SETUP: Create the three types of retrievers

# 1. Vector Retriever (Semantic Search)


In [20]:
print("Setting up Vector Retriever...")
embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")
vectorstore = Chroma.from_documents(
    documents=documents,
    embedding=embedding_model,
    collection_metadata={"hnsw:space": "cosine"}
)
vector_retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

Setting up Vector Retriever...


In [21]:
# Test semantic search
# test_query = "How much did the purchase cost?"
test_query = "electric car company financials"
# test_query = "rocket development for space travel"  

print(f"Testing: '{test_query}'")
test_docs = vector_retriever.invoke(test_query)
for doc in test_docs[:2]:
    print(f"Found: {doc.page_content}")

Testing: 'electric car company financials'
Found: Tesla Tesla Tesla financial quarterly results improved significantly.
Found: Tesla Tesla Tesla financial quarterly results improved significantly.


# 2. BM25 Retriever (Keyword Search)


In [22]:
# 2. BM25 Retriever (Keyword Search)
print("Setting up BM25 Retriever...")
bm25_retriever = BM25Retriever.from_documents(documents)
bm25_retriever.k = 3

Setting up BM25 Retriever...


In [23]:
# Test exact keyword matching
# test_query = "7.5 billion"
# test_query = "Cybertruck"
test_query = "Tesla"

print(f"Testing: '{test_query}'")
test_docs = bm25_retriever.invoke(test_query)
for doc in test_docs[:2]:
    print(f"Found: {doc.page_content}")

Testing: 'Tesla'
Found: Tesla Tesla Tesla financial quarterly results improved significantly.
Found: Tesla reported strong quarterly results. Tesla continues to lead in electric vehicles. Tesla announced new manufacturing facilities.


# 3. Hybrid Retriever (Combination)

In [24]:
#  3. Hybrid Retriever (Combination)
print("Setting up Hybrid Retriever...")
hybrid_retriever = EnsembleRetriever(
    retrievers=[vector_retriever, bm25_retriever],
    weights=[0.5, 0.5]  # Equal weight to vector and keyword search
)

print("Setup complete!\n")

Setting up Hybrid Retriever...
Setup complete!



In [31]:
# Query 1: Mixed semantic and exact terms
# Vector search understands "purchase cost" semantically
# BM25 search finds exact "7.5 billion" 
# Hybrid should combine both strengths for best result
test_query = "purchase cost 7.5 billion"

test_docs = hybrid_retriever.invoke(test_query)
for i, doc in enumerate(test_docs[:3], 1):
    print(f"{i}. {doc.page_content}")
print()

print("Query 1 shows how hybrid finds exact financial info using both semantic understanding and keyword matching")


1. Microsoft acquired GitHub for 7.5 billion dollars in 2018.
2. Cybertruck reservations exceeded company expectations.
3. Microsoft is a large technology company with global operations.

Query 1 shows how hybrid finds exact financial info using both semantic understanding and keyword matching


In [ ]:
# Query 2: Semantic concept + specific product name  

# Vector search understands "electric vehicle manufacturing"
# BM25 search finds exact "Cybertruck"
# Hybrid gets the best of both worlds

test_query = "electric vehicle manufacturing Cybertruck"

test_docs = hybrid_retriever.invoke(test_query)

for i, doc in enumerate(test_docs[:3], 1):
    print(f"{i}. {doc.page_content}")
print()

print("Query 2 demonstrates combining product-specific terms with broader concepts")



1. Tesla Cybertruck production ramp begins in 2024.
2. Cybertruck reservations exceeded company expectations.
3. Tesla reported strong quarterly results. Tesla continues to lead in electric vehicles. Tesla announced new manufacturing facilities.


In [32]:
# Query 3: Where neither alone would be perfect

# "Company performance" is semantic, "Tesla" is exact keyword
# Hybrid should find the most relevant Tesla performance info

test_query = "company performance Tesla"

test_docs = hybrid_retriever.invoke(test_query)
for i, doc in enumerate(test_docs[:3], 1):
    print(f"{i}. {doc.page_content}")
print()

print("Query 3 shows how hybrid handles mixed semantic/keyword queries better than either approach alone")

1. Tesla Tesla Tesla financial quarterly results improved significantly.
2. Tesla reported strong quarterly results. Tesla continues to lead in electric vehicles. Tesla announced new manufacturing facilities.
3. Cybertruck reservations exceeded company expectations.

Query 3 shows how hybrid handles mixed semantic/keyword queries better than either approach alone
